In [1]:
# !pip install pyarxiv
# !pip install pybtex
# !pip install dropbox
# !pip install tqdm
# !pip install pycolors2
# !pip install pdf2image

# Libraries

In [2]:
from pathlib import Path
from bib_handling_code.processbib import read_bibfile
from bib_handling_code.processbib import save_to_file
import pandas as pd
import os
import numpy as np
from ast import literal_eval

# Imports

## DIAG bib

In [3]:
path_diag_bib = os.path.join('..', 'diag.bib')
path_output_diag_bib = os.path.join('..', 'diag_taverne.bib')

In [4]:
diag_bib_raw = read_bibfile(None, path_diag_bib) # I changed the code in such a way that IF I give a second argument, it uses the second argument as a full path

## SS match data

### multiple_bibkeys_maybe_multiple_ss_ids
- bibkey : many
- ss_id: none, one, many 

In [5]:
df_matches_many = pd.read_csv('script_data/TEMP_LOOKUP_multiple_bibkeys_maybe_multiple_ss_ids.csv')
# df_matches_multiple.head(3)

In [20]:
bibkeys_many = []
for str_list in list(df_matches_many['up80_bibkeys']):
    bibkeys_many.extend(literal_eval(str_list))
len(bibkeys_many)

227

In [22]:
# df_matches_many.head()

### matches_single_bibkey_with_none_or_many_ss_ids
Filename is misleading right now, during further investigation we removed the 1-to-1 matches because these were fine
- bibkey: one
- ss_id: none, one, many

In [7]:
df_matches_one_to_none_or_many = pd.read_csv('script_data/TEMP_LOOKUP_matches_single_bibkey_with_none_one_or_many_ss_ids.csv')
# df_matches_one = df_matches_one[~df_matches_one['bibkey'].isin(many_bibkeys)]
# df_matches_one = df_matches_one[[col for col in df_matches_one.columns if col != "Unnamed: 0"]]

In [8]:
print(len(df_matches_one_to_none_or_many))
df_matches_one_to_none_or_many.head()

121


,bibkey,type,title,authors,ss_id,all_ss_ids,comment,year
0,Abel19,article,"Computational pathology definitions, best prac...","Abels, Esther and Pantanowitz, Liron and Aeffn...",NaN,NaN,NaN,2019.0
1,Amga20,article,Report on computational assessment of Tumor In...,Mohamed Amgad and and Elisabeth Specht Stovgaa...,b4c4c3dc91d42114023b0575c3e2273b87446ff7,"['69999230b02054b82254684a73bb8a4c83878d28', '...",[ss_ids: same DOI] title matching: single bibk...,2020.0
2,Anto21,article,The Medical Segmentation Decathlon,Michela Antonelli and Annika Reinke and Spyrid...,NaN,NaN,NaN,2021.0
3,Anto22,article,The {Medical} {Segmentation} {Decathlon,"Antonelli, Michela and Reinke, Annika and Baka...",979a9f247700d00ff2c3f0612d5eb001379f93c8,"['979a9f247700d00ff2c3f0612d5eb001379f93c8', '...",[ss_ids: same DOI] multiple doi matches ['979a...,2022.0
4,Ares18,article,iW-Net}: an automatic and minimalistic interac...,Guilherme Aresta and Colin Jacobs and Teresa A...,NaN,NaN,NaN,2018.0


In [9]:
bibkeys_one_to_none_or_many = df_matches_one_to_none_or_many['bibkey']

In [18]:
bibkeys_one_to_none_or_many

0        Abel19
1        Amga20
2        Anto21
3        Anto22
4        Ares18
         ...   
116      Graa23
117    Alves22c
118    Bosma23a
119      Spro23
120      Vend23
Name: bibkey, Length: 121, dtype: object

### df one-to-one

In [10]:
df_one_to_one_matches = pd.read_csv('script_data/df_matches.csv')
df_one_to_one_matches = df_one_to_one_matches[~df_one_to_one_matches['type'].isin(['conference', 'book', 'inbook', 'mastersthesis', 'phdthesis', 'patent', 'misc'])]
df_one_to_one_matches = df_one_to_one_matches[df_one_to_one_matches['comment'].isin(['title matching: single bibkey, single ss_id', 'single doi match'])]
df_one_to_one_matches['all_ss_ids'] = df_one_to_one_matches['ss_id'].apply(lambda x: [x])
df_one_to_one_matches.to_csv('script_data/df_one_to_one_matches.csv', index=False)
df_one_to_one_matches = pd.read_csv('script_data/df_one_to_one_matches.csv')

In [11]:
df_one_to_one_matches.head()

,bibkey,type,title,authors,ss_id,all_ss_ids,comment
0,Abra08a,article,Evaluation of a system for automatic detection...,M. D. Abr\`amoff and M. Niemeijer and M. S. A....,0445a1a0aa30c9ca41a8ab31463d617b849a0814,['0445a1a0aa30c9ca41a8ab31463d617b849a0814'],"title matching: single bibkey, single ss_id"
1,Adri10,article,Echocardiographic screening results in patient...,Miraude E A P M Adriaensen and Maarten J M Cra...,3da4ab4d50211eb7a0148c4ed74729b2d4e30ba9,['3da4ab4d50211eb7a0148c4ed74729b2d4e30ba9'],"title matching: single bibkey, single ss_id"
2,Adri11,article,Radiological evidence of lymphangioleiomyomato...,M. E A P M Adriaensen and C. M. Schaefer-Proko...,691c90e68eab1dcae5861fba0eab73ba626e6777,['691c90e68eab1dcae5861fba0eab73ba626e6777'],"title matching: single bibkey, single ss_id"
3,Adri11b,article,Mature fat cells in the myocardium of patients...,Miraude E A P M Adriaensen and Matthijs F M va...,c738da78dc1fb336a8dff5a8047059d52a6de939,['c738da78dc1fb336a8dff5a8047059d52a6de939'],"title matching: single bibkey, single ss_id"
4,Adri11c,article,Focal fatty areas in the myocardium of patient...,Miraude E A P M Adriaensen and Harm H H Fering...,9fabf1bc1c126702a051ec56474af2d1518ad0ba,['9fabf1bc1c126702a051ec56474af2d1518ad0ba'],single doi match


# Add ss_ids to DAIG bib

#### Logic:
Loop over entries:
- Skip Journal strings
- Look up entry key (bibkey) in df_matches_one
    - If bibkey not in df, skip
- Check if there are SS matches for this bibkey
    - If so, add this row's 'all_ss_ids' column to the entry

In [12]:
diag_bib_raw[2000].key

'Mann12a'

In [13]:
diag_bib_raw[2000].fields

{'author': '{R. Mann and R. Mus and C. Geppert and C. Frentz and N. Karssemeijer and HJ. Huisman and B. Platel}',
 'title': '{Initial maximum slope of the contrast enhancement versus time curve for dynamic evaluation of breast lesions on ultrafast breast MRIs}',
 'booktitle': 'ECR',
 'year': '{2012}',
 'url': '{http://ipp.myesr.org/esr/ecr2012/index.php?v=sessiond&seid=247&ippwwwsid=5jnho9nq3agreqirle8hkru2l2}',
 'abstract': '{Purpose: Modern viewsharing sequences allow dynamic whole breast MRI at diagnostic spatial resolution within 5 seconds, showing the inflow of contrast in breast abnormalities. We evaluate the maximum slope of initial contrast enhancement as a heuristic parameter for differentiating benign from malignant disease and compare its value to classic 3 timepoint analysis. Methods and Materials: We employed a bi-temporal breast MRI protocol on a 3 T MRI using a 16 channel bilateral breast coil. High-resolution VIBE acquisitions (0.8*0.8*1.0 mm, temporal resolution 94s) w

In [14]:
for entry in diag_bib_raw:
    ### skip Journal strings
    if entry.type == 'string':
        continue
    
    # Get data
    bibkey = entry.key
    match_row = df_one_to_one_matches[df_one_to_one_matches['bibkey'] == bibkey]
    
    # Double checks
    if len(match_row) == 0:
        continue
    assert len(match_row) == 1, 'multiple rows should not be possible in this DF'
    match_row = match_row.iloc[0]
    
    # If there is a match
    if pd.notna(match_row['all_ss_ids']):
        ## SS ID
        # If the field is already filled, check if the same
        if 'ss_id' in entry.fields:
            if not entry.fields['ss_id'] == '{' + match_row['ss_id'] + '}':
                print('ss_id data not the same\n', bibkey, entry.fields['ss_id'], '{' + match_row['ss_id'] + '}')
                break
        # Else fill field
        else: 
            entry.fields['ss_id'] = '{' + match_row['ss_id'] + '}'
            
        ## SS ID
        # If the field is already filled, check if the same
        if 'all_ss_ids' in entry.fields:
            if not entry.fields['all_ss_ids'] == '{' + match_row['all_ss_ids'] + '}':
                print('all_ss_ids data not the same\n', bibkey, entry.fields['all_ss_ids'], '\n', match_row)
                break
        # Else fill field
        else: 
            entry.fields['all_ss_ids'] = '{' + match_row['all_ss_ids'] + '}'
    
    
#     if entry.type == 'phdthesis':
#         entry.fields['journal'] = '{PhD thesis}'
#     if entry.type == 'mastersthesis':
#         entry.fields['journal'] = '{Master thesis}'
#     else:
#         pass

In [15]:
count = 0
for entry in diag_bib_raw:
    if 'ss_id' in entry.fields:
        count += 1
print(count)

917


# Save new diag_taverne.bib

Here its important to be aware that withing the processbib.py file from the bib handling code, there is a hardcoded 'allowed_fields' variable on top of the file. Only fields with names with in this list can be saved to the diag.bib

In [16]:
path_output_diag_bib = os.path.join('script_data', 'diag_ss.bib')

In [17]:
save_to_file(diag_bib_raw, None, path_output_diag_bib)